# Multi-Container Endpoints Custom Container Example

#### Container 1: [spacy](https://spacy.io/)
#### Container 2: [textblob](https://textblob.readthedocs.io/en/dev/)

In [1]:
import boto3
from sagemaker import get_execution_role

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')
account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name
role = get_execution_role()

## Container 1: Spacy

In [2]:
%%sh

# Name of algo -> ECR
algorithm_name=mce-spacy-container

cd container-spacy

#make serve executable
chmod +x NER/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Region, defaults to us-west-2
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  22.02kB
Step 1/11 : FROM python:3.8
 ---> 5c4350efb04f
Step 2/11 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python3          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> eb9fd35375b2
Step 3/11 : RUN wget https://bootstrap.pypa.io/get-pip.py && python3 get-pip.py &&     pip install flask gevent gunicorn &&         rm -rf /root/.cache
 ---> Using cache
 ---> 84058e0d6278
Step 4/11 : RUN pip install spacy
 ---> Using cache
 ---> 4810a52bed63
Step 5/11 : RUN python -m spacy download en
 ---> Using cache
 ---> 8fd0cb27a2c9
Step 6/11 : LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true
 ---> Using cache
 ---> 01163a42ea92
Step 7/11 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> e69e45ca212f
Step 8/11 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 70a780eaf91e
Step 9/11 : ENV PATH="/opt/program:${PATH}"
 ---> U

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [3]:
spacy_mce_container = '{}.dkr.ecr.{}.amazonaws.com/mce-spacy-container:latest'.format(account_id, region)
spacy_mce_container

'474422712127.dkr.ecr.us-east-1.amazonaws.com/mce-spacy-container:latest'

## TextBlob Push

In [4]:
%%sh

# Name of algo -> ECR
algorithm_name=mce-textblob-container

cd container-textblob

#make serve executable
chmod +x Sentiment/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Region, defaults to us-west-2
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  23.04kB
Step 1/10 : FROM python:3.8
 ---> 5c4350efb04f
Step 2/10 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python3          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> eb9fd35375b2
Step 3/10 : RUN wget https://bootstrap.pypa.io/get-pip.py && python3 get-pip.py &&     pip install flask gevent gunicorn &&         rm -rf /root/.cache
 ---> Using cache
 ---> 84058e0d6278
Step 4/10 : RUN pip install textblob
 ---> Using cache
 ---> 52f026decb4e
Step 5/10 : LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true
 ---> Using cache
 ---> 10cf9f647cb9
Step 6/10 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 74cc93a19690
Step 7/10 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 4a244d07e518
Step 8/10 : ENV PATH="/opt/program:${PATH}"
 ---> Using cache
 ---> 6759414d97ed
Step 9/10 : COPY Sentiment /opt/program
 ---> Us

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [5]:
textblob_mce_container = '{}.dkr.ecr.{}.amazonaws.com/mce-textblob-container:latest'.format(account_id, region)
textblob_mce_container

'474422712127.dkr.ecr.us-east-1.amazonaws.com/mce-textblob-container:latest'

# SageMaker Client Setup

- [SageMaker Client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model): Model, Endpoint Config, and Endpoint Creation
- [SageMaker RunTime Client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime.html#SageMakerRuntime.Client.invoke_endpoint): Endpoint Invocation/Testing

## Model Creation

In [6]:
container1 = {'Image':spacy_mce_container,'ContainerHostname': 'spacyContainer'}
container2 = {'Image':textblob_mce_container,'ContainerHostname': 'textblobContainer'}
inferenceExecutionConfig = {'Mode': 'Direct'}   

In [7]:
import boto3
from time import gmtime, strftime
sm_client = boto3.Session().client('sagemaker')

#define our MCE in the containers param
model_name = "multi-container-ep-custom" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
response = sm_client.create_model(
              ModelName = model_name,
              InferenceExecutionConfig = inferenceExecutionConfig,
              ExecutionRoleArn = role,
              Containers = [container1, container2])

### Endpoint Config Creation

In [8]:
epc_config_name = "multi-container-epc" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
endpoint_config = sm_client.create_endpoint_config(
    EndpointConfigName=epc_config_name,
    ProductionVariants=[
        {
            "VariantName": "prod",
            "ModelName": model_name,
            "InitialInstanceCount": 1,
            "InstanceType": "ml.r5.24xlarge",
        },
    ],
)

### Endpoint Creation

In [9]:
endpoint_name = "mce-custom" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
endpoint = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=epc_config_name
)

In [10]:
import time
describe_endpoint = sm_client.describe_endpoint(EndpointName=endpoint_name)

endpoint_status = describe_endpoint["EndpointStatus"]

while endpoint_status != "InService":
    print("Current endpoint status is: {}, Trying again...".format(endpoint_status))
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    endpoint_status = resp["EndpointStatus"]

print("Endpoint status changed to 'InService'")

Current endpoint status is: Creating, Trying again...
Current endpoint status is: Creating, Trying again...
Current endpoint status is: Creating, Trying again...
Current endpoint status is: Creating, Trying again...
Current endpoint status is: Creating, Trying again...
Current endpoint status is: Creating, Trying again...
Endpoint status changed to 'InService'


### Test Spacy Container

In [13]:
import json
content_type = "application/json"
request_body = {"input": "This is a test with NER in America with Amazon and Microsoft in Seattle, writing random stuff."}

#Serialize data for endpoint
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)

#Endpoint invocation
response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=payload,
    TargetContainerHostname="spacyContainer")

#Parse results
result = json.loads(response['Body'].read().decode())
result

{'output': [['NER', 'ORG'],
  ['America', 'GPE'],
  ['Amazon', 'ORG'],
  ['Microsoft', 'ORG'],
  ['Seattle', 'GPE']]}

### Test Textblob Container

In [14]:
import json
content_type = "application/json"
request_body = {"input": "This is a test with NER in America with Amazon and Microsoft in Seattle, writing random stuff."}

#Serialize data for endpoint
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)

#Endpoint invocation
response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=payload,
    TargetContainerHostname="textblobContainer")

#Parse results
result = json.loads(response['Body'].read().decode())
result

{'Polarity': -0.5, 'Subjectivity': 0.5}